In [1]:
import arepo_run as arun
import matplotlib.pylab as plt
import numpy as np
import gadget
# import yt
# import cmasher

#Some constants
gamma    = 5./3
unit_m   = 1.989e43
unit_v   = 1.e5
unit_l   = 3.09567758e21
unit_t   = unit_l/unit_v
unit_rho = unit_m/unit_l**3



In [2]:
kpc_rs = ((6.64e-8*100*unit_m)/(100*(70*unit_v/(unit_l*1e3))**2))**(1/3)/(7*unit_l) #scale radius

In [3]:
#loading a proxy snap
# o  = arun.Run(snappath='/home/c5046973/agn/gasCloudTest/arepo_t/output_bolalt', snapbase="snap_")
# s  = o.loadSnap(snapnum=10)

In [4]:
# ---------- user inputs ----------
# Option A: you have halo mass M_delta (Msun) and want r_delta then r_s via c
M_delta = 100        # e.g. 1e12   # set to None if you don't have M_delta
Delta = 200           # common choices: 200 or virial
c = 7                # concentration (choose per mass/redshift)

# Option B: you already know r_s directly (in same length units as r)
r_s = kpc_rs             # set to your scale radius (e.g., in kpc or arbitrary units)

# If using physical units to compute rho_crit:
H0 = 70.0             # km/s/Mpc, change if needed
G = 6.64e-8    # cm^3 g^-1 s^-2
# Msun = 1.98847e30     # kg
# Mpc = 3.085677581e22  # meters

# ---------- helper functions ----------
def rho_crit(H0_km_s_Mpc):
    # returns critical density in cgs
    H0 = H0_km_s_Mpc*unit_v/(unit_l*1e3)   # s^-1
    return 3.0 * H0**2 / (8*np.pi*G)

def delta_c_from_c(c, Delta=200):
    return (Delta/3.0) * c**3 / (np.log(1.0+c) - c/(1.0+c))

def r_delta_from_M(M_delta, Delta=200, H0_km_s_Mpc=70.0):
    rho_c = rho_crit(H0_km_s_Mpc)   # g/cm^3
    M = M_delta * unit_m
    r_m = (3.0 * M / (4.0*np.pi * Delta * rho_c))**(1.0/3.0)
    return r_m / unit_l  # return in kpc

# ---------- compute values ----------
if M_delta is not None:
    r_delta_Mpc = r_delta_from_M(M_delta, Delta=Delta, H0_km_s_Mpc=H0)
    r_s = r_delta_Mpc / c
    print(f"r_{Delta} = {r_delta_Mpc:.4f} Mpc, r_s = {r_s:.4f} Mpc (c={c})")
else:
    # assume r_s already set (units must match r array units)
    print(f"Using provided r_s = {r_s} (user units) and c = {c} (if you want to compute delta_c)")

# characteristic overdensity
delta_c = delta_c_from_c(c, Delta=Delta)
print("delta_c =", delta_c)

# critical density (useful if you want absolute densities)
rho_c = rho_crit(H0)
rho_c_10Msun_per_kpc3 = rho_c * (unit_l**3) / unit_m
print(f"rho_crit = {rho_c_10Msun_per_kpc3:.3e} 10Msun / kpc^3 (for H0={H0} km/s/Mpc)")

# ---------- NFW profile function ----------
def rho_nfw(r, r_s, delta_c, rho_crit):
    x = r / r_s
    return rho_crit * delta_c / (x * (1.0 + x)**2)





r_200 = 205.7234 Mpc, r_s = 29.3891 Mpc (c=7)
delta_c = 18985.285607780104
rho_crit = 1.371e-08 10Msun / kpc^3 (for H0=70.0 km/s/Mpc)


In [16]:
def calculate_R_free(beta, tau, b, L_AGN, rho_0):
    """
    Calculate R_free (free expansion radius) based on equation (15).
    
    Parameters:
    -----------
    beta : float
        Beta parameter (dimensionless)
    tau : float
        Optical depth parameter (dimensionless)
    b : float
        Parameter b (dimensionless)
    L_AGN : float
        AGN luminosity in erg s^(-1)
    n_0 : float
        Number density in cm^(-3)
    
    Returns:
    --------
    R_free : float
        Free expansion radius in parsecs (pc)
    
    Formula:
    --------
    R_free ≈ 10 * (β / 0.1)^(-1) * (τ / b)^(1/2) * 
             (L_AGN / (10^45 erg s^(-1)))^(1/2) * 
             (n_0 / cm^(-3))^(-1/2) pc
    """
    c = 3e10  # speed of light in cm s^(-1)
    
    M_dot_W = (tau * L_AGN) / (beta * c**2)
    v_W = beta * c
    
    t_free = np.sqrt((3 / (4 * np.pi * b)) * (M_dot_W / (rho_0 * v_W**3)))
    R_free = v_W * t_free / (3.086e18)  # Convert cm to pc
    
    return R_free, t_free

In [27]:
#calculation of number density in cm^-3
m_p = 1.67e-24# mass of proton in g
mu = 0.6 # mean molecular weight
c = 3e5# speed of light in kms^-1

rho_0 = 1e-3*unit_rho #fiducial value of density in g/cm^3
calculate_R_free(beta = 0.017, tau = 1, b = 1, L_AGN = 1e45, rho_0 = rho_0)#value of free radius in pc

(69.22216232117563, 418861946908.1333)

In [28]:
calculate_R_free(beta = 5000/c, tau = 1, b = 1, L_AGN = 1e45, rho_0 = rho_0)[1]/(3.15e13)#value of free time in Myears

0.013834411732165777

In [20]:
def calculate_R_free_from_params(beta, tau, b, L_AGN, n_H, mu=0.6, khi=0.76, 
                                  L_ref=1e45, n_ref=1.0, beta_ref=0.1):
    """
    Calculate R_free (free expansion radius) from parameters.
    
    Parameters:
    -----------
    beta : float
        Beta parameter (dimensionless)
    tau : float
        Optical depth (dimensionless)
    b : float
        Parameter b (dimensionless)
    L_AGN : float
        AGN luminosity in erg s^(-1)
    rho_0 : float
        Ambient gas density in g cm^(-3)
    mu : float, optional
        Mean molecular weight (default: 0.6)
    khi : float, optional
        Hydrogen fraction (default: 0.76)
    L_ref : float, optional
        Reference luminosity in erg s^(-1) (default: 1e45)
    n_ref : float, optional
        Reference number density in cm^(-3) (default: 1.0)
    beta_ref : float, optional
        Reference beta parameter (default: 0.1)
    
    Returns:
    --------
    R_free : float
        Free expansion radius in parsecs (pc)
    """
    m_p = 1.67e-24  # mass of proton in g
    
    # Calculate number density
    #n_h = khi*rho_0 / m_p
    n_0 = n_H / (mu * khi)
    
    # Calculate R_free using equation (15)
    R_free = 10.0 * (beta / beta_ref)**(-1) * (tau / b)**(0.5) * \
             (L_AGN / L_ref)**(0.5) * (n_0 / n_ref)**(-0.5)
    
    return R_free

calculate_R_free_from_params(beta = 0.017, tau = 1, b = 1, L_AGN = 1e45, n_H = 1.0)#value of free radius in pc

39.722218861492074

In [12]:
def calculate_t_free(beta, tau, b, L_AGN, rho_0, v_W):
    """
    Calculate t_free (free expansion timescale).
    
    Parameters:
    -----------
    beta : float
        Beta parameter (dimensionless)
    tau : float
        Optical depth (dimensionless)
    b : float
        Parameter b (dimensionless)
    L_AGN : float
        AGN luminosity in erg s^(-1)
    rho_0 : float
        Ambient gas density in g cm^(-3)
    v_W : float
        Wind velocity in cm s^(-1)
    
    Returns:
    --------
    t_free : float
        Free expansion timescale in seconds
    """
    c = 3e5  # speed of light in cm s^(-1)
    
    M_dot_W = (tau * L_AGN) / (beta * c**2)
    
    t_free = np.sqrt((3 / (4 * np.pi * b)) * (M_dot_W / (rho_0 * v_W**3)))
    
    return t_free